<a href="https://colab.research.google.com/github/dyanni3/Blog/blob/master/SENN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 
##1.   Creating the Autoencoder for basis concepts using MNIST as example 



First going to create the simplest possible autoencoder. It's a fully connected layer as the encoder and decoder

The way to train the autoencoder is to make the y_train data be exactly the x_train data. i.e. you want to do a reconstruction, so you want the output to equal the input.


### Sparsity:

The encoded images do not look very sparse. For the purpose of explainable AI we don't only want a low reconstruction error, we also want the encoded units to represent *concepts*. In order for that to be the case it's best if a given input highly activates only few encoded units. For example for the mnist dataset it would be very natural for the autoencoder to learn ten concepts (one for each digit) just like a human. Then activation of the encoded units would be sparse (mostly just one unit activated for any given input image). Of course, to actually achieve this we'd likely need to force the encoded dimension to be exactly 10. This is pretty unlikely for an arbitrary dataset. Presumably we could discover structure in the data during EDA (e.g. by doing some clustering), but anyway the principle is that sparsity *should* give us reasonably coherent concepts - to the extent that the data allow.



![Autoencoder Architecture](https://softmattermachines.files.wordpress.com/2019/01/autoencoder.png)

In [0]:
##### making the autoencoder with tensorflow ######

#imports
import numpy as np
from matplotlib import pyplot as plt
from keras.layers import Dense, Input, Dropout
from keras.models import Model
from keras import datasets
from keras import regularizers

#load data
mnist = datasets.mnist
(x_train, _), (x_test, _) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

#reshape data and prep
x_train = np.reshape(x_train,(len(x_train),784))
x_test = np.reshape(x_test,(len(x_test),784))
encoding_dim_1 = 164
encoding_dim_2 = 10

#set up models
input_img = Input(shape = (784,))
encoded = Dense(encoding_dim_1, activation = 'relu')(input_img)
encoded = Dropout(.3)(encoded)
encoded = Dense(encoding_dim_2,activation='relu',
               activity_regularizer=regularizers.l1(2.5e-6))(encoded)
decoded = Dense(encoding_dim_1, activation = 'relu')(encoded)
#decoded = Dropout(.3)(decoded)
decoded = Dense(784,activation='sigmoid')(decoded)
autoencoder = Model(input_img,decoded)
encoder = Model(input_img,encoded)
decoder_layer_1 = autoencoder.layers[-2]
decoder_layer_2 = autoencoder.layers[-1]
encoded_input = Input(shape = (encoding_dim_2,))
decoder = Model(encoded_input, decoder_layer_2(decoder_layer_1(encoded_input)))

#compile and fit
autoencoder.compile(optimizer='adadelta', loss = 'binary_crossentropy')
autoencoder.fit(x_train, x_train,
                epochs=1,
                batch_size=256,
                shuffle=True,
                verbose=2,
                validation_data=(x_test, x_test))


In [0]:
#predict and plot
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)
fig,ax = plt.subplots(nrows=3,ncols=10)
for j in range(3):
  for i in range(10):
    #plot the input
    ax[0][i].imshow(x_test[i].reshape(28,28),cmap=plt.cm.gray)
    #plot the reconstructed image
    ax[1][i].imshow(decoded_imgs[i].reshape(28,28),cmap = plt.cm.gray)
    #plot the encoded image
    ax[2][i].bar(np.arange(10),encoded_imgs[i].reshape(10,))

Not too shabby! Note that we don't need the reconstructions to be perfect, because in the end we're going to pair these with generalized weights to make predictions. What we need from the reconstructions is for them to be human understandable, each basis concept should be something interpretable!

Still, since the input are images, we can do much better for this data set with a convolutional autoencoder. Something like the following architecture.

![conv_autoencoder](https://softmattermachines.files.wordpress.com/2019/01/conv_autoencoder.png)

In [0]:
 from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K

input_img = Input(shape=(28, 28, 1))  # adapt this if using `channels_first` image data format

x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
encoded = Dense(10,activation='relu',
               activity_regularizer=regularizers.l1(2.5e-6))(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

"""encoder = Model(input_img, encoded)
encoded_input = Input(shape=(10,))
decoder_layers = autoencoder.layers[7:]
x = decoder_layers[0](encoded_input)
for i in range(1,6):
  x = decoder_layers[i](x)
decoder_output = decoder_layers[6](x)
decoder = Model(encoded_input,decoder_output)"""

from keras.datasets import mnist
import numpy as np

(x_train, _), (x_test, _) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))  # adapt this if using `channels_first` image data format
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))  # adapt this if using `channels_first` image data format

In [0]:
history = autoencoder.fit(x_train, x_train,
                epochs=1,
                batch_size=128,
                shuffle=True,
                verbose=1,
                validation_data=(x_test, x_test))

In [0]:
from matplotlib import pyplot as plt
plt.plot(history.history['val_loss'],label = "validation")
plt.plot(history.history['loss'],label = "training")
plt.legend()

In [0]:
decoded_imgs = autoencoder.predict(x_test)

n = 10
plt.figure(figsize=(20, 4))
for i in range(1,n):
    # display original
    ax = plt.subplot(2, n, i)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [0]:
encoder = Model(input_img, encoded)

In [0]:
encoded_preds = encoder.predict(x_test)

In [0]:
fig, axes = plt.subplots(figsize=(20,2),nrows=1, ncols=10)
for i in range(10):
  for j in range(4):
    for k in range(4):
      axes[i].bar(np.arange(10),encoded_preds[i,j,k])

In [0]:
encoded_preds.shape